# LMER in rpy2

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from CrossOverExperiment import CrossOverExperiment, RCT

import rpy2

# Install LMER packages (THIS TAKES ABOUT 3~5 minutes)
packnames = ('lme4', 'lmerTest', 'emmeans', "geepack")
from rpy2.robjects.packages import importr, data
from rpy2.robjects.vectors import StrVector

utils = importr("utils")
utils.chooseCRANmirror(ind=1)

import pandas as pd
import rpy2.robjects as robjects
from rpy2.ipython.ggplot import image_png
from rpy2.robjects import pandas2ri# Defining the R script and loading the instance in Python
r = robjects.r

from statsmodels.api import MixedLM


import numpy as np
import pandas as pd

pandas2ri.activate()

### I had to install all these in this order before getting lme4 to work with rpy2

## R imports

In [82]:
lme4 = importr('lme4')
lmertest = importr('lmerTest')

base = importr('base')

stats = importr("stats")

## prepare some test data

In [31]:
crossover_experiment = CrossOverExperiment(subjects=15,periods=6,base_effect=2,mu=1,tau=0.2,carryover=0.1,alpha_sd=2,epsilon_sd=1,error_type="whatever, statsmodels supports only idiosyncratic")
crossover_experiment.params

{'Subjects': 15,
 'Periods': 6,
 'Base Effect': 2,
 'Mu': 1,
 'Tau': 0.2,
 'Carryover Effect': 0.1,
 'Alpha Standard Deviation': 2,
 'Epsilon Standard Deviation': 1,
 'Error Structure': 'whatever, statsmodels supports only idiosyncratic'}

In [32]:
(y0,y1),sub,T,t,cO = crossover_experiment.generate_data(return_for_t=False)

In [78]:
exog = pd.DataFrame(np.array([np.ones(len(T)),T,t,cO,sub]).T)#.reset_index()
exog.columns = ["len_T", "T", "t", "c0", "sub"]

In [69]:
endog = pd.DataFrame(y0)
endog.columns = ["y0"]

In [79]:
endogexog = pd.concat([endog,exog], axis=1)

## try 2 ways

In [80]:
fm1 = lme4.lmer('y0 ~ (1|sub) + len_T + T + t + c0', data = endogexog)
print(base.summary(fm1))

R[write to console]: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Linear mixed model fit by REML ['lmerMod']
Formula: y0 ~ (1 | sub) + len_T + T + t + c0
   Data: 
structure(list(y0 = c(2.83089032345401, 1.39934989553122, 2.47476391970883,  
1.76987801883385, 1.64351434211287, 2.48411842880599, 5.63568855005975,  
5.96005189503025, 4.98576721774086, 5.45958923593027, 5.45231160460251,  
4.51150622184518, 1.34596182764986, 1.02259349638255, 1.11321222992043,  
2.08687037623647, 0.528790982436147, 1.48663652936439, 1.37898568557442,  
1.99508634583664, 2.68271867927307, 3.53683576570774, 1.15027961425394,  
2.11234722992763, 3.93368739241317, 4.45541594948876, 3.16892831932006,  
5.40659848666555, 4.37131590388065, 4.17287736973744, 3.81020046729932,  
3.18514249563821, 2.69272603378357, 1.65474286344309, 4.98290321041136,  
6.30714578815309, 2.47465045923392, 1.1189609952358, 2.17922358560418,  
1.58566915310148, 2.86895998169964, 3.49344743562697, 2.08719401734007,  
0.052770106190118, 0.59528185753668, 2.58940103715813, 0.196016615119232,  
2.589924

In [92]:
fm2 = lmertest.lmer('y0 ~ (1|sub) + len_T + T + t + c0', data = endogexog)
print(base.summary(fm2))

R[write to console]: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: "y0 ~ (1|sub) + len_T + T + t + c0"
   Data: 
structure(list(y0 = c(2.83089032345401, 1.39934989553122, 2.47476391970883,  
1.76987801883385, 1.64351434211287, 2.48411842880599, 5.63568855005975,  
5.96005189503025, 4.98576721774086, 5.45958923593027, 5.45231160460251,  
4.51150622184518, 1.34596182764986, 1.02259349638255, 1.11321222992043,  
2.08687037623647, 0.528790982436147, 1.48663652936439, 1.37898568557442,  
1.99508634583664, 2.68271867927307, 3.53683576570774, 1.15027961425394,  
2.11234722992763, 3.93368739241317, 4.45541594948876, 3.16892831932006,  
5.40659848666555, 4.37131590388065, 4.17287736973744, 3.81020046729932,  
3.18514249563821, 2.69272603378357, 1.65474286344309, 4.98290321041136,  
6.30714578815309, 2.47465045923392, 1.1189609952358, 2.17922358560418,  
1.58566915310148, 2.86895998169964, 3.49344743562697, 2.08719401734007,  
0.052770106190118, 0.59528185753668, 2.58

## try to extract p-values

In [91]:
fm2 = lmertest.lmer('y0 ~ (1|sub) + len_T + T + t + c0', data = endogexog)
print(stats.anova(fm2))

R[write to console]: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Type III Analysis of Variance Table with Satterthwaite's method
      Sum Sq Mean Sq NumDF  DenDF F value  Pr(>F)  
T     0.4783  0.4783     1 72.065  0.4505 0.50426  
t     7.1845  7.1845     1 72.056  6.7667 0.01126 *
c0    2.4954  2.4954     1 72.315  2.3502 0.12963  
len_T                                              
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



In [138]:
fm2anova = stats.anova(fm2)

In [141]:
fm2anova.rx2("Pr(>F)") # p_values

array([0.50425741, 0.0112643 , 0.12962892,        nan])